In [1]:
import pandas as pd
import numpy as np
import os
pd.set_option('display.max_rows', 500)
pd.set_option('display.max_columns', 500)
import tqdm
import matplotlib.pyplot as plt
import seaborn as sns
import matplotlib
matplotlib.rcParams['axes.unicode_minus'] = False
pd.options.mode.chained_assignment = None  # default='warn'
plt.rcParams['font.family'] = 'Malgun Gothic'
font = {'size': 16}
matplotlib.rc('font', **font)
tqdm.tqdm.pandas()
import multiprocess as mp

In [2]:
mp.__version__

'0.70.12.2'

In [3]:
np.__version__

'1.21.5'

In [4]:
%%time
main_data_df = pd.read_csv('./Prepped Data/main_data_new_building_merge_land_specs_20211228.csv', dtype={1:'str', 9:'Int64', 10:'Int64'})
#main_data_df = next(main_data_df)
main_data_df['계약날짜'] = pd.to_datetime(main_data_df['계약날짜'])
main_data_df = main_data_df.sort_values(['시군구', '본번', '부번', '대장구분명', '건축년도', '부동산유형', '계약날짜'])
print(main_data_df.shape)
main_data_df.head()

(5006836, 33)
Wall time: 25.5 s


,시군구,지번,본번,부번,건물/단지명,거래유형,전용면적(㎡),거래/보증금(만원),월세(만원),층,건축년도,도로명,부동산유형,가격범위,건물단지명_키워드,브랜드,대장구분명,지목,x좌표,y좌표,계약날짜,계약날짜일수_2006년기준,지번주소,토지일련번호,지목명,토지면적,용도지역명1,용도지역명2,토지이동상황,지형높이,지형형상,도로접면,데이터기준일자
0,서울특별시 강남구 개포동,12,12,0,성원대치2단지아파트,매매,33.18,15900,0,1,1992,개포로109길 9,아파트,14000~16000,성원대치단지,기타,일반,대,206607.181143,443586.061143,2006-01-03,2,서울특별시 강남구 개포동 12,5864,대,121040.4,제3종일반주거지역,지정되지않음,아파트,평지,부정형,광대소각,2021-07-29
1,서울특별시 강남구 개포동,12,12,0,삼익대청아파트,매매,39.53,24800,0,5,1992,개포로109길 21,아파트,24000~26000,삼익대청,기타,일반,대,206607.181143,443586.061143,2006-01-07,6,서울특별시 강남구 개포동 12,5864,대,121040.4,제3종일반주거지역,지정되지않음,아파트,평지,부정형,광대소각,2021-07-29
2,서울특별시 강남구 개포동,12,12,0,성원대치2단지아파트,매매,39.53,24700,0,12,1992,개포로109길 9,아파트,24000~26000,성원대치단지,기타,일반,대,206607.181143,443586.061143,2006-01-09,8,서울특별시 강남구 개포동 12,5864,대,121040.4,제3종일반주거지역,지정되지않음,아파트,평지,부정형,광대소각,2021-07-29
3,서울특별시 강남구 개포동,12,12,0,성원대치2단지아파트,매매,49.86,33300,0,11,1992,개포로109길 9,아파트,32000~34000,성원대치단지,기타,일반,대,206607.181143,443586.061143,2006-01-09,8,서울특별시 강남구 개포동 12,5864,대,121040.4,제3종일반주거지역,지정되지않음,아파트,평지,부정형,광대소각,2021-07-29
4,서울특별시 강남구 개포동,12,12,0,성원대치2단지아파트,매매,39.53,24700,0,4,1992,개포로109길 9,아파트,24000~26000,성원대치단지,기타,일반,대,206607.181143,443586.061143,2006-01-10,9,서울특별시 강남구 개포동 12,5864,대,121040.4,제3종일반주거지역,지정되지않음,아파트,평지,부정형,광대소각,2021-07-29


In [5]:
main_data_df['지번주소'] = main_data_df['시군구'] + ' ' + main_data_df['지번'] + ' ' + main_data_df['건축년도'].astype('str') + ' ' + main_data_df['부동산유형'] + ' ' + main_data_df['거래유형']

In [6]:
main_data_df['거래유형'].value_counts()

매매    2065298
전세    1982727
월세     958811
Name: 거래유형, dtype: int64

In [7]:
main_data_df['distance_x'] = np.nan
main_data_df['distance_y'] = np.nan
main_data_df['distance'] = np.nan

In [8]:
main_data_df['거래/보증금(만원)'].describe()

count    5.006836e+06
mean     3.056585e+04
std      3.083686e+04
min      0.000000e+00
25%      1.230000e+04
50%      2.300000e+04
75%      3.900000e+04
max      1.850000e+06
Name: 거래/보증금(만원), dtype: float64

In [9]:
main_data_df['계약날짜'].std()

Timedelta('1497 days 16:54:01.654263040')

In [10]:
main_data_df['계약날짜'].describe()

c:\users\max6296\ml\lib\site-packages\ipykernel_launcher.py:1: FutureWarning: Treating datetime data as categorical rather than numeric in `.describe` is deprecated and will be removed in a future version of pandas. Specify `datetime_is_numeric=True` to silence this warning and adopt the future behavior now.
  """Entry point for launching an IPython kernel.


count                 5006836
unique                   5814
top       2020-06-13 00:00:00
freq                     3202
first     2006-01-01 00:00:00
last      2021-12-01 00:00:00
Name: 계약날짜, dtype: object

In [11]:
dir(main_data_df['x좌표'])

['T',
 '_AXIS_LEN',
 '_AXIS_ORDERS',
 '_AXIS_REVERSED',
 '_AXIS_TO_AXIS_NUMBER',
 '_HANDLED_TYPES',
 '__abs__',
 '__add__',
 '__and__',
 '__annotations__',
 '__array__',
 '__array_priority__',
 '__array_ufunc__',
 '__array_wrap__',
 '__bool__',
 '__class__',
 '__contains__',
 '__copy__',
 '__deepcopy__',
 '__delattr__',
 '__delitem__',
 '__dict__',
 '__dir__',
 '__divmod__',
 '__doc__',
 '__eq__',
 '__finalize__',
 '__float__',
 '__floordiv__',
 '__format__',
 '__ge__',
 '__getattr__',
 '__getattribute__',
 '__getitem__',
 '__getstate__',
 '__gt__',
 '__hash__',
 '__iadd__',
 '__iand__',
 '__ifloordiv__',
 '__imod__',
 '__imul__',
 '__init__',
 '__init_subclass__',
 '__int__',
 '__invert__',
 '__ior__',
 '__ipow__',
 '__isub__',
 '__iter__',
 '__itruediv__',
 '__ixor__',
 '__le__',
 '__len__',
 '__long__',
 '__lt__',
 '__matmul__',
 '__mod__',
 '__module__',
 '__mul__',
 '__ne__',
 '__neg__',
 '__new__',
 '__nonzero__',
 '__or__',
 '__pos__',
 '__pow__',
 '__radd__',
 '__rand__',
 '__r

In [12]:
def year_circular_sine_func(x):
    return math.sin(2*math.pi*((x)/365))

def year_circular_cosine_func(x):
    return math.cos(2*math.pi*((x)/365))

In [13]:
main_data_df['건축년도'].value_counts()

2003    300832
2002    284446
2004    278358
2005    177366
2008    176479
1988    175953
2001    169452
2000    165053
1999    156144
2012    142601
2011    134697
1997    132844
2013    131171
1993    130794
1992    127954
2014    127171
2006    124343
2016    124269
1998    120227
2007    116580
1986    113139
1994    111121
1996    108876
2015    107877
1991    100427
2009     98544
1995     98523
2010     97996
1987     96700
2017     95362
2018     85221
1989     78447
2019     76580
1990     65562
2020     51993
1983     42051
1985     34569
1984     33876
1982     32637
1978     31615
1979     25976
1980     22445
2021     21611
1981     21451
1976     10919
1977     10566
1974      9556
1971      8139
1975      7942
1973      3598
1970      2096
1972      1555
1969      1141
1968       877
1965       383
1966       287
1961       124
1967       123
1956        94
1964        39
1963        29
1962         8
1959         8
1960         7
1940         5
1925         5
1955      

In [14]:
'''main_data_df = main_data_df.sample(frac=0.01).reset_index(drop=True)
print(main_data_df.shape)
main_data_df.head()'''

'main_data_df = main_data_df.sample(frac=0.01).reset_index(drop=True)\nprint(main_data_df.shape)\nmain_data_df.head()'

In [15]:
def prac_func(x):
    print(x.columns)
    display(x.head())
    print(type(x))
    
    raise ValueError('hmm')

In [16]:
main_data_df['건축년도'].mean()

2001.4401963635319

In [17]:
def temp_func(df):
    import pandas as pd
    import numpy as np
    
    df = df.copy()
    
    ranges_list = [1000]
    
    numerical_cols = ['전용면적(㎡)', '거래/보증금(만원)', '월세(만원)', '층', '건축년도', 'x좌표', 'y좌표', '계약날짜', '계약날짜일수_2006년기준', '토지면적']
    
    numerical_description_features_list = [
        'mean', 'std', 'min', 'quantile(0.25)', 'quantile(0.5)', 'quantile(0.75)', 'max', 'sum', 'sem', 'skew', 'var'
    ]
    
    date_offset_months = 6    
    
    all_numerical_feature_cols = []
    for target_range in ranges_list:
        for col in numerical_cols:
            for f in numerical_description_features_list:
                col_name = '최근{}개월_'.format(date_offset_months) + str(target_range) + '미터이내_' + col + '_' + f
                df[col_name] = np.nan
                all_numerical_feature_cols.append(col_name)
    
    def numerical_description_feature_creator(x, y):
    
        if y == 'mean':
            return x.mean()
        elif y == 'std':
            return x.std()
        elif y == 'min':
            return x.min()
        elif y == 'quantile(0.25)':
            return x.quantile(0.25)
        elif y == 'quantile(0.5)':
            return x.quantile(0.5)
        elif y == 'quantile(0.75)':
            return x.quantile(0.75)
        elif y == 'max':
            return x.max()
        elif y == 'sum':
            return x.sum()
        elif y == 'sem':
            return x.sem()
        elif y == 'skew':
            return x.skew()
        elif y == 'var':
            return x.var()
        else:
            raise ValueError('feature name error')
     
    def groupby_function(x):
        # x is a groupby dataframe

        # vars
        coor_x = x['x좌표'].iloc[0]
        coor_y = x['y좌표'].iloc[0]
        addr = x['지번주소'].iloc[0]
        house_type = x['지번주소'].iloc[0].split(' ')[-2]
        trade_type = x['지번주소'].iloc[0].split(' ')[-1]
        print(trade_type)
        built_year = int(x['건축년도'].iloc[0])
        print(built_year)
        unique_contract_dates = x['계약날짜'].unique()

        #
        df['distance_x'] = df['x좌표'] - coor_x
        df['distance_y'] = df['y좌표'] - coor_y
        df['distance'] = np.sqrt(df['distance_x']**2 + df['distance_y']**2)
        
        for target_range in ranges_list:

            range_df = df[
                (df['distance'] <= target_range)
                &(df['지번주소'] != addr)
                &(df['부동산유형'] == house_type)
                &(df['거래유형'] == trade_type)
                &(df['건축년도'] <= built_year)
                &(df['건축년도'] > (built_year - 5))
            ]
            
            #
            if range_df.shape[0] == 0:
                target_range_cols = [col for col in all_numerical_feature_cols if str(target_range) in col]
                
                i_list = []
                for i in range(x.shape[0]):
                    j_list = []
                    for j in range(len(target_range_cols)):
                        j_list.append(np.nan)
                    
                    i_list.append(j_list)
                
                nan_df = pd.DataFrame(np.array(i_list), columns=target_range_cols)
                
                return nan_df
            
            #
            else:
                
                for contract_date in unique_contract_dates:
                    contract_date = pd.to_datetime(contract_date)
                    
                    original_df = x[x['계약날짜'] == contract_date]
                    
                    contract_date_df = range_df[
                        (range_df['계약날짜'] <= contract_date)
                        &(range_df['계약날짜'] > (contract_date - pd.DateOffset(months=date_offset_months)))
                    ]
                    
                    if contract_date_df.shape[0] == 0:
                        target_range_cols = [col for col in all_numerical_feature_cols if str(target_range) in col]
                        
                        i_list = []
                        for i in range(original_df.shape[0]):
                            j_list = []
                            for j in range(len(target_range_cols)):
                                j_list.append(np.nan)

                            i_list.append(j_list)

                        nan_df = pd.DataFrame(np.array(i_list), columns=target_range_cols)
                        
                        x.loc[original_df.index, target_range_cols] = nan_df
                        #return nan_df
                    
                    
                    #
                    else:
                        
                        
                        for num_col in numerical_cols:
                            
                            for num_feat in numerical_description_features_list:
                                
                                feat_col_name = '최근{}개월_'.format(date_offset_months) + str(target_range) + '미터이내_' + num_col + '_' + num_feat
                                
                                try:
                                    x.loc[original_df.index, feat_col_name] = numerical_description_feature_creator(contract_date_df[num_col], num_feat)
                                except:
                                    x.loc[original_df.index, feat_col_name] = np.nan
                        
        return x[all_numerical_feature_cols]
    
    df[all_numerical_feature_cols] = df.groupby(['시군구', '지번', '건축년도', '부동산유형', '거래유형']).apply(groupby_function).reset_index(drop=True)
    
    return df

In [18]:
def parallelize(df, func, num_processors=6):
    #data_split = np.array_split(data, num_processors)
    
    data_split = []
    unique_addrs = df['지번주소'].unique()
    divided = int(len(unique_addrs) / num_processors)
    for i in range(num_processors):
        if i < (num_processors - 1):
            picked_addrs = unique_addrs[divided*i:divided*(i+1)]
        else:
            picked_addrs = unique_addrs[divided*i:]
        data_split.append(df[df['지번주소'].isin(picked_addrs)])
            
    pool = mp.Pool(num_processors)
    concat_df = pd.concat(pool.map(func, data_split))
    pool.close()
    pool.join()
    
    concat_df = concat_df.sort_values(['시군구', '본번', '부번', '건축년도', '계약날짜']).reset_index(drop=True)
    
    return concat_df

In [19]:
%%time
completed_df = parallelize(main_data_df, temp_func)
print(completed_df.shape)
completed_df.head()

(5006836, 146)
Wall time: 12h 19min 46s


,시군구,지번,본번,부번,건물/단지명,거래유형,전용면적(㎡),거래/보증금(만원),월세(만원),층,건축년도,도로명,부동산유형,가격범위,건물단지명_키워드,브랜드,대장구분명,지목,x좌표,y좌표,계약날짜,계약날짜일수_2006년기준,지번주소,토지일련번호,지목명,토지면적,용도지역명1,용도지역명2,토지이동상황,지형높이,지형형상,도로접면,데이터기준일자,distance_x,distance_y,distance,최근6개월_1000미터이내_전용면적(㎡)_mean,최근6개월_1000미터이내_전용면적(㎡)_std,최근6개월_1000미터이내_전용면적(㎡)_min,최근6개월_1000미터이내_전용면적(㎡)_quantile(0.25),최근6개월_1000미터이내_전용면적(㎡)_quantile(0.5),최근6개월_1000미터이내_전용면적(㎡)_quantile(0.75),최근6개월_1000미터이내_전용면적(㎡)_max,최근6개월_1000미터이내_전용면적(㎡)_sum,최근6개월_1000미터이내_전용면적(㎡)_sem,최근6개월_1000미터이내_전용면적(㎡)_skew,최근6개월_1000미터이내_전용면적(㎡)_var,최근6개월_1000미터이내_거래/보증금(만원)_mean,최근6개월_1000미터이내_거래/보증금(만원)_std,최근6개월_1000미터이내_거래/보증금(만원)_min,최근6개월_1000미터이내_거래/보증금(만원)_quantile(0.25),최근6개월_1000미터이내_거래/보증금(만원)_quantile(0.5),최근6개월_1000미터이내_거래/보증금(만원)_quantile(0.75),최근6개월_1000미터이내_거래/보증금(만원)_max,최근6개월_1000미터이내_거래/보증금(만원)_sum,최근6개월_1000미터이내_거래/보증금(만원)_sem,최근6개월_1000미터이내_거래/보증금(만원)_skew,최근6개월_1000미터이내_거래/보증금(만원)_var,최근6개월_1000미터이내_월세(만원)_mean,최근6개월_1000미터이내_월세(만원)_std,최근6개월_1000미터이내_월세(만원)_min,최근6개월_1000미터이내_월세(만원)_quantile(0.25),최근6개월_1000미터이내_월세(만원)_quantile(0.5),최근6개월_1000미터이내_월세(만원)_quantile(0.75),최근6개월_1000미터이내_월세(만원)_max,최근6개월_1000미터이내_월세(만원)_sum,최근6개월_1000미터이내_월세(만원)_sem,최근6개월_1000미터이내_월세(만원)_skew,최근6개월_1000미터이내_월세(만원)_var,최근6개월_1000미터이내_층_mean,최근6개월_1000미터이내_층_std,최근6개월_1000미터이내_층_min,최근6개월_1000미터이내_층_quantile(0.25),최근6개월_1000미터이내_층_quantile(0.5),최근6개월_1000미터이내_층_quantile(0.75),최근6개월_1000미터이내_층_max,최근6개월_1000미터이내_층_sum,최근6개월_1000미터이내_층_sem,최근6개월_1000미터이내_층_skew,최근6개월_1000미터이내_층_var,최근6개월_1000미터이내_건축년도_mean,최근6개월_1000미터이내_건축년도_std,최근6개월_1000미터이내_건축년도_min,최근6개월_1000미터이내_건축년도_quantile(0.25),최근6개월_1000미터이내_건축년도_quantile(0.5),최근6개월_1000미터이내_건축년도_quantile(0.75),최근6개월_1000미터이내_건축년도_max,최근6개월_1000미터이내_건축년도_sum,최근6개월_1000미터이내_건축년도_sem,최근6개월_1000미터이내_건축년도_skew,최근6개월_1000미터이내_건축년도_var,최근6개월_1000미터이내_x좌표_mean,최근6개월_1000미터이내_x좌표_std,최근6개월_1000미터이내_x좌표_min,최근6개월_1000미터이내_x좌표_quantile(0.25),최근6개월_1000미터이내_x좌표_quantile(0.5),최근6개월_1000미터이내_x좌표_quantile(0.75),최근6개월_1000미터이내_x좌표_max,최근6개월_1000미터이내_x좌표_sum,최근6개월_1000미터이내_x좌표_sem,최근6개월_1000미터이내_x좌표_skew,최근6개월_1000미터이내_x좌표_var,최근6개월_1000미터이내_y좌표_mean,최근6개월_1000미터이내_y좌표_std,최근6개월_1000미터이내_y좌표_min,최근6개월_1000미터이내_y좌표_quantile(0.25),최근6개월_1000미터이내_y좌표_quantile(0.5),최근6개월_1000미터이내_y좌표_quantile(0.75),최근6개월_1000미터이내_y좌표_max,최근6개월_1000미터이내_y좌표_sum,최근6개월_1000미터이내_y좌표_sem,최근6개월_1000미터이내_y좌표_skew,최근6개월_1000미터이내_y좌표_var,최근6개월_1000미터이내_계약날짜_mean,최근6개월_1000미터이내_계약날짜_std,최근6개월_1000미터이내_계약날짜_min,최근6개월_1000미터이내_계약날짜_quantile(0.25),최근6개월_1000미터이내_계약날짜_quantile(0.5),최근6개월_1000미터이내_계약날짜_quantile(0.75),최근6개월_1000미터이내_계약날짜_max,최근6개월_1000미터이내_계약날짜_sum,최근6개월_1000미터이내_계약날짜_sem,최근6개월_1000미터이내_계약날짜_skew,최근6개월_1000미터이내_계약날짜_var,최근6개월_1000미터이내_계약날짜일수_2006년기준_mean,최근6개월_1000미터이내_계약날짜일수_2006년기준_std,최근6개월_1000미터이내_계약날짜일수_2006년기준_min,최근6개월_1000미터이내_계약날짜일수_2006년기준_quantile(0.25),최근6개월_1000미터이내_계약날짜일수_2006년기준_quantile(0.5),최근6개월_1000미터이내_계약날짜일수_2006년기준_quantile(0.75),최근6개월_1000미터이내_계약날짜일수_2006년기준_max,최근6개월_1000미터이내_계약날짜일수_2006년기준_sum,최근6개월_1000미터이내_계약날짜일수_2006년기준_sem,최근6개월_1000미터이내_계약날짜일수_2006년기준_skew,최근6개월_1000미터이내_계약날짜일수_2006년기준_var,최근6개월_1000미터이내_토지면적_mean,최근6개월_1000미터이내_토지면적_std,최근6개월_1000미터이내_토지면적_min,최근6개월_1000미터이내_토지면적_quantile(0.25),최근6개월_1000미터이내_토지면적_quantile(0.5),최근6개월_1000미터이내_토지면적_quantile(0.75),최근6개월_1000미터이내_토지면적_max,최근6개월_1000미터이내_토지면적_sum,최근6개월_1000미터이내_토지면적_sem,최근6개월_1000미터이내_토지면적_skew,최근6개월_1000미터이내_토지면적_var
0,서울특별시 강남구 개포동,12,12,0,성원대치2단지아파트,매매,33.18,15900,0,1,1992,개포로109길 9,아파트,14000~16000,성원대치단지,기타,일반,대,206607.181143,443586.061143,2006-01-03,2,서울특별시 강남구 개포동 12 1992 아파트 매매,5864,대,121040.4,제3종일반주거지역,지정되지않음,아파트,평지,부정형,광대소각,2021-07-29,20350.746879,-5292.12664,21027.589089,33.107500,14.786866,15.45,20.3900,34.8875,45.57125,48.730,198.645,6.036713,-0.172739,218.651397,8333.333333,6282.250128,2000.0,2750.0,7500.0,13750.0,16000.0,50000.0,2564.717875,0.249792,3.946667e+07,34.166667,14.634434,15.0,22.50,37.5,45.00,50.0,205.0,5.974483,-0.329695,214.166667,3.500000,0.83666,2.0,3.25,4.0,4.0,4.0,21.0,0.341565,-1.536722,0.7,2011.83

In [20]:
%%time
completed_df.to_csv('./Prepped Data/main_data_range_implement_20220103.csv', index=False)

Wall time: 3min 27s
